In [ ]:
import requests
import pandas as pd
import numpy as np

def crawl_financial(url, year, season, exchange):
    r = requests.post(url, {
        'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'TYPEK':exchange,
        'year':str(year),
        'season':str(season),
    })

    r.encoding = 'utf8'
    return pd.read_html(r.text, header=None)


def financial_statement(year, season, exchange='sii', type='綜合損益彙總表'):
     # 一些參數：TYPEK => 市場別 sii>上市 otc>上櫃 rotc>興櫃 pub>公開發行 year => 年度 season => 季別
    if year >= 1000:
        year -= 1911

    if type == '綜合損益彙總表':
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb04'
        dfs = crawl_financial(url, year, season, exchange)
        return pd.concat(dfs[1:], axis=0, sort=False).set_index(pd.concat(dfs[1:], axis=0, sort=False).columns[0]).apply(lambda s: pd.to_numeric(s, errors='ignore'))
    elif type == '資產負債彙總表':
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb05'
        dfs = crawl_financial(url, year, season, exchange)
        return pd.concat(dfs[1:], axis=0, sort=False).set_index(pd.concat(dfs[1:], axis=0, sort=False).columns[0]).apply(lambda s: pd.to_numeric(s, errors='ignore'))
    elif type == '營益分析彙總表':
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb06'
        dfs = crawl_financial(url, year, season, exchange)
        
        for i, df in enumerate(dfs):
            df.columns = df.iloc[0]
            dfs[i] = df.iloc[1:]

            df = df[df['公司代號'] != '公司代號']
            df = df[~df['公司代號'].isnull()]
            if any(df.columns.str.contains("合計：")):
                df = df.loc[:, ~df.columns.str.contains("合計：")]
            return df
    else:
        return ValueError('type does not match')

financial_statement(2020, 2, 'rotc',type='營益分析彙總表')

In [ ]:
# strMode = 2: 上市公司  4: 上櫃公司

import requests
import pandas as pd

res = requests.get("http://isin.twse.com.tw/isin/C_public.jsp?strMode=2")

df = pd.read_html(res.text)[0]

# 設定column名稱
df.columns = df.iloc[0]
# 刪除第一行
df = df.iloc[1:]

df = df.dropna(thresh=3, axis=0).dropna(thresh=3, axis=1)

df = df.set_index('有價證券代號及名稱')
df